# Tutorial 15: Saving and Loading Warmup States

This tutorial walks you through the process of saving and loading initial states of traffic in Flow. For a large number of the networks provided in this repository, the onset of congestion does not occur as soon as the network is initialized, but instead may require several hundreds simulation steps for dense traffic to build up and ultamitely result in formation of traffic shockwaves or other forms of congestion. In order to subvert the computational cost of repeatedly initializing a network to a desired state, this tutorial describes how initial states can be generated, saved, and then used as the start state for any new simulation or training environment (see the figure below).

<img src="img/init_state_example.png">

The remainder of this tutorial is organized as follows:

* Section 1 describes how initial states can be saved and describes the format of the file that are generated.
* Section 2 describes the process of reloading a previously saved state.
* Section 3 describes how individual vehicles in the saved state can at runtime be replaced with different agents possessing different types of controllers.

## 1. Saving Initial Traffic States

In this section, we describe how initial states can saved, and the format of the generate files.

We will demonstrate the method of generating initial states on a single lane highway network. The network is initialized with no vehicles and an inflow rate of 2000 veh/hr. The script below defines the flow-specific parameters for this network (see `tutorial01_sumo.ipynb` for more details on defining these parameters).

**Note**: For the time being, this features only works if at least one vehicle type is specified as "human", see line 17 in the script below.

In [ ]:
from flow.controllers import IDMController
from flow.core.params import EnvParams
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import InFlows
from flow.core.params import VehicleParams
from flow.core.params import SumoParams
from flow.networks import HighwayNetwork
from flow.envs import TestEnv
from flow.utils.registry import make_create_env


def get_flow_params(render=True):
    """Return the flow-specific parameters for a single-lane highway."""
    # Create the vehicles class.
    vehicles = VehicleParams()
    vehicles.add("human", acceleration_controller=(IDMController, {}))

    # Create the inflows class.
    inflows = InFlows()
    inflows.add(
        veh_type="human",
        edge="highway_0",
        vehs_per_hour=2000,
        depart_lane="free",
        depart_speed=24.1)

    # Fill in the remainder of the flow_params dict, see: 
    # examples/exp_configs/non_rl/highway_single.py
    return dict(
        exp_tag='highway-single',
        env_name=TestEnv,
        network=HighwayNetwork,
        simulator='traci',
        env=EnvParams(),
        sim=SumoParams(
            sim_step=0.2,
            render=render,
            use_ballistic=True,
        ),
        net=NetParams(
            inflows=inflows,
            additional_params={
                "length": 1000,
                "lanes": 1,
                "speed_limit": 30,
                "num_edges": 2,
                "use_ghost_edge": True,
                "ghost_speed_limit": 6.0,
                "boundary_cell_length": 300,
            }
        ),
        veh=vehicles,
        initial=InitialConfig(),
    )

In order to save the initial state of a given simulation, two parameters must be defined within the `SumoParams` object (saved under "sim" in flow_params). These parameters, represented below, are `save_state_time` and `save_state_file`, and represent that time at which the simulation state should be saved and where it should be saved to, respectively.

In [ ]:
flow_params = get_flow_params()

# specify the time (in seconds) that the initial state should be saved
flow_params["sim"].save_state_time = 200

# specify the path where the initial state should be saved to
flow_params["sim"].save_state_file = "initial_state.xml"

Once these variables have been defined, we simply run a simulation of the environment as would be done in any other situation, ensuring that the time horizon **matches of exceeds** the assigned `save_state_time` value. Once this time has been reached, a new xml file will be generated in the path you assigned above. This xml value will contain within it the speeds, positions, IDs, etc... of all vehicles that were within the network at the time the snapshot was taken. In the next section, we will describe how this file can be exploited to initial a simulation with a warm-started initial state.

In [ ]:
# Specify a horizon to match the save time.
horizon = int(flow_params["sim"].save_state_time / flow_params["sim"].sim_step)

# Run the environment until the state is saved.
env_creator, _ = make_create_env(flow_params)
env = env_creator()
env.reset()
for _ in range(horizon):
    env.step(None)

# Terminate the environment.
env.terminate()

## 2. Loading Previously Saved States

To initialize a new simulation with the snapshot we had saved in the previous section, we now define a new variable under `SumoParams`: `load_state`. This variable takes the same path you had assigned under `save_state_file` before, however, this path will now be read the initial configuration and vehicles and reassign them to an otherwise empty network.

In [ ]:
flow_params = get_flow_params()

# specify the path to the initial state
flow_params["sim"].load_state = "initial_state.xml"

Having defined the `load_state` value above, we can now validate that the initial state of the new simulation does in fact match the previously saved state. The below script runs a simple reset() operation and renders the first image of the simulation. If this operation was succeful, you should notice that the configuration of vehicles matches the final states vehicles were in after the last code snippet in Section 1 was run.

In [ ]:
# Initialize and reset the environment.
env_creator, _ = make_create_env(flow_params)
env = env_creator()
env.reset()

# Terminate the environment.
env.terminate()

## 3. Adding Vehicle Types

Vehicles at the start of the simulation are, by default, as a vehicle type of "human" when initialized from a save state. As such no vehicles are treated as RL vehicle (or vehicles of any other type) at the start of a simulation. In the subsections below, we introduce two techniques for introducing vehicle of varying types to a simulation once the network has been initialized. These techniques account for the introduction of new types from inflowing lanes as well as the modification of the types of existing vehicles at the start of a simulation.

### 3.1 Adding inflow vehicles of multiple types

We begin by demonstrated how RL vehicles can be introduced to a network through inflows from various edges. We begin by recreating the original flow_params object, and this time introduce a new vehicle type to the `Vehicles` class, in this case of type "rl" with an `RLController` class.

In [ ]:
# Get the base flow params.
flow_params = get_flow_params()

# Add the initial state (see Section 2).
flow_params["sim"].load_state = "initial_state.xml"

# Add a vehicle type for RL agents.
from flow.controllers import RLController
flow_params["veh"].add("rl", acceleration_controller=(RLController, {}))

Once this vehicle type exists, an inflow for vehicles of this type can simply be incorporated to the `InFlows` class as is done in Section 1 of this tutorial for the "human" type.

In [ ]:
# Replace the inflow with inflows of two types:
#  - human, with an inflow rate of 1000 veh/hr
#  - rl, with an inflow rate of 1000 veh/hr
inflows = InFlows()
inflows.add(
    veh_type="human",
    edge="highway_0",
    vehs_per_hour=1000,
    depart_speed=24.1)
inflows.add(
    veh_type="rl",
    edge="highway_0",
    vehs_per_hour=1000,
    depart_speed=24.1)

# Replace the inflow in the flow parameters.
flow_params["net"].inflows = inflows

If you run the simulation now, every other vehicle that enters will be red, indicating that it is a vehicle of the type "rl".

In [ ]:
# Initialize and run the environment.
env_creator, _ = make_create_env(flow_params)
env = env_creator()
env.reset()
for _ in range(1000):
    env.step(None)

# Terminate the environment.
env.terminate()

### 3.2 Replace vehicle type in the initial state

We have managed to introduce RL vehicles to the network from the inflow edge, but what if we would like to treat a subset of the existing vehicle as RL vehicles as well? To do so, we will use the `set_vehicle_type` method within the `env.k.vehicle` object. Before we demonstrate this, we first demonstrate that, unmodified, vehicles within the network are in fact solely human drivers. We do so in the script below by simply reinitializaing the saved state and reading the number of human and RL vehicles.

In [ ]:
# Get the base flow params (we will not render the environment this time).
flow_params = get_flow_params(render=False)

# Add the initial state (see Section 2).
flow_params["sim"].load_state = "initial_state.xml"

# Add a vehicle type for RL agents (see Section 3.1).
from flow.controllers import RLController
flow_params["veh"].add("rl", acceleration_controller=(RLController, {}))

# Initialize the environment.
env_creator, _ = make_create_env(flow_params)
env = env_creator()
env.reset()

print("Number of humans: {}".format(env.k.vehicle.num_vehicles - env.k.vehicle.num_rl_vehicles))
print("Number of RL:     {}".format(env.k.vehicle.num_rl_vehicles))
print("RL IDs:           {}".format(env.k.vehicle.get_rl_ids()))

Let us now say we would like to choose one of the above vehicle and from the current iteration update it's vehicle type such that it falls under the type "rl". To do so, we can simply call the `env.k.vehicle.set_vehicle_type` and pass to it as variables the name of the vehicle whose type we'd like to change and the type we'd like to modify it to. We demonstrate this below for a random vehicle, and replace it's type with "rl".

In [ ]:
import random

# Pick a vehicle to change to an RL vehicle.
vehicle_to_change = random.choice(env.k.vehicle.get_ids())

print("Replacing vehicle:", vehicle_to_change)

# Use the set_vehicle_type method to replace the vehicle type to the RL type.
env.k.vehicle.set_vehicle_type(vehicle_to_change, "rl")

If we now try to reference the features that define the types of vehicles (in this case human and rl vehicles), we can see that the vehicle type of vehicle we tried to change about has in facted been switched to "rl". This vehicle will now not act as a "human" type vehicle in runtime.

In [ ]:
print("Number of humans: {}".format(env.k.vehicle.num_vehicles - env.k.vehicle.num_rl_vehicles))
print("Number of RL:     {}".format(env.k.vehicle.num_rl_vehicles))
print("RL IDs:           {}".format(env.k.vehicle.get_rl_ids()))